# epic 4(sh)

Als een key user kan ik voor een contact met weinig transacties een lookalike met veel transacties identificeren. Ik kan ook een clustering maken van contactpersonen die qua jobinhoud, type bedrijf, voorkeuren en (verwacht) gedrag

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
ENV_URL = os.path.join(os.getcwd(), '../.env')
load_dotenv(ENV_URL)

DWH_NAME = os.environ.get('DWH_NAME')
SERVER_NAME = os.environ.get('SERVER_NAME')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')

URL = f'mssql+pymssql://{DB_USER}:{DB_PASSWORD}@{SERVER_NAME}/{DWH_NAME}'
URL_LOCAL = f'mssql+pyodbc://{SERVER_NAME}/{DWH_NAME}?trusted_connection=yes&driver=ODBC+Driver+17 for SQL Server'

engine = create_engine(URL_LOCAL)
conn = engine.connect()

In [3]:
def create_query(table_name, columns, condition=None):

    query = f"SELECT "

    for i, column in enumerate(columns):
        if i == 0:
            query += f"[{column}]"
        else:
            query += f", [{column}]"
    
    query += f" FROM [{DWH_NAME}].[dbo].[{table_name}]"
    
    if condition:
        query += f" WHERE {condition}"

    return query

### 1 Alles van account selecteren

In [4]:
acc_cols = ['accountID', 'plaats','subregio','ondernemingsaard','ondernemingstype','activiteitNaam']
# account conditie
acc_condition = "provincie = 'Oost-Vlaanderen'"
# create query
acc_query = create_query('DimAccount', acc_cols, acc_condition)
# read sql
df_account = pd.read_sql(acc_query, conn)
df_account.shape

(4124, 6)

### 2 Mergen van contact en account

In [5]:
contact_cols = ['contactID', 'accountID', 'functietitel','functieNaam']

contact_query = create_query('DimContact', contact_cols)
df_contact = pd.read_sql(contact_query, conn)

df_contact['functietitel'] = df_contact['functietitel'].str.lower()

df_contact['functieNaam'] = df_contact['functieNaam'].str.lower()
df_contact.shape

(194192, 4)

In [6]:
accounts_merged = pd.merge(df_contact, df_account, on='accountID', how='inner')
accounts_merged.shape

(61885, 9)

### 3 Mergen van account en afspraak

In [7]:
afspraak_cols = ['accountID', 'keyphrases']

afspraak_query = create_query('DimAfspraak', afspraak_cols)

df_afspraak = pd.read_sql(afspraak_query, conn)
df_afspraak.shape

(7167, 2)

In [8]:
acc_con_afs = pd.merge(accounts_merged, df_afspraak, on='accountID', how='inner')
acc_con_afs.shape

(63673, 10)

### 4 Mergen van Campagne en account

In [9]:
campagne_cols = ['campagneID','campagneType','campagneNaam','campagneSoort']

campagne_query = create_query('DimCampagne', campagne_cols)

df_campagne = pd.read_sql(campagne_query, conn)
df_campagne.shape

(4101, 4)

In [10]:
factInschrijving_cols = ['campagneID','contactID']

factInschrijving_query = create_query('FactInschrijving', factInschrijving_cols)

df_factInschrijving = pd.read_sql(factInschrijving_query, conn)
df_factInschrijving.shape

(78790, 2)

In [11]:
camp_fact = pd.merge(df_campagne, df_factInschrijving, on='campagneID', how='inner')
camp_fact.shape

(78790, 5)

In [12]:
df = pd.merge(acc_con_afs, camp_fact, on='contactID', how='inner')
df = df.drop_duplicates(subset=['contactID','campagneID'], keep='first')
df = df[['contactID','plaats','subregio','ondernemingsaard','ondernemingstype','activiteitNaam','campagneID','campagneType','campagneNaam','campagneSoort','keyphrases','functietitel','functieNaam']]
df.shape

(12963, 13)

In [13]:
df.head()

,contactID,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,campagneID,campagneType,campagneNaam,campagneSoort,keyphrases,functietitel,functieNaam
0,0542DA63-2C64-ED11-9561-6045BD895B5A,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline,"workshop, plantages, student, pakket, onli...","teamleider graduaatsopleidingen enw bmg, artev...",medewerker
2,09E85092-AF88-EC11-93B0-6045BD91D362,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,C6D72260-E451-EC11-8C62-000D3ABFCF4A,Netwerkevenement,OV-NW-VokaUpdate-Big Refresh-Voorjaar 2022,Online,"workshop, plantages, student, pakket, onli...",student,medewerker
5,153B9FE0-68BA-E811-80F4-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,7835D8C0-F488-E811-80F3-001DD8B72B61,Netwerkevenement,OV-NW-Verderkijkers 2018-Think Customer,Offline,"workshop, plantages, student, pakket, onli...",marketingverantwoordelijke,medewerker marketing
6,1FE13719-73A4-EC11-983F-00224884C0D3,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline,"workshop, plantages, student, pakket, onli...",community manager & coördinator postgraduaat o...,medewerker
7,1FE13719-73A4-EC11-983F-00224884C0D3,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,4C8FF159-145C-EC11-8F8F-000D3A2BCF4B,Infosessie,OV-JO Breakfastclub april 2022,Offline,"workshop, plantages, student, pakket, onli...",community manager & coördinator postgraduaat o...,medewerker


## Data Cleaning

In [14]:
# combine all the data 
df3 = df.copy()
df2 = df
df2['data'] =df[df.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
print(df['data'].head())

0    Gent,Gent,Diensten,Onderwijs,Overige industrie...
2    Gent,Gent,Diensten,Onderwijs,Overige industrie...
5    Gent,Gent,Diensten,Onderwijs,Overige industrie...
6    Gent,Gent,Diensten,Onderwijs,Overige industrie...
7    Gent,Gent,Diensten,Onderwijs,Overige industrie...
Name: data, dtype: object


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized= vectorizer.fit_transform(df2['data'])

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

#duurt 2m38s
similarities = cosine_similarity(vectorized)

In [17]:
print(similarities)


[[1.         0.75678747 0.77093425 ... 0.10022297 0.10022297 0.11514155]
 [0.75678747 1.         0.77791118 ... 0.08427498 0.08427498 0.09958592]
 [0.77093425 0.77791118 1.         ... 0.12019049 0.12019049 0.15217125]
 ...
 [0.10022297 0.08427498 0.12019049 ... 1.         0.875      0.846254  ]
 [0.10022297 0.08427498 0.12019049 ... 0.875      1.         0.846254  ]
 [0.11514155 0.09958592 0.15217125 ... 0.846254   0.846254   1.        ]]


In [36]:
df_similarities = pd.DataFrame(similarities,columns=df['contactID'],index=df['contactID']).reset_index()
# df_scores = pd.DataFrame(similarities,columns=df['contactID'],index=df['contactID'])
df_similarities.head()
# df_scores.head()

contactID,contactID,0542DA63-2C64-ED11-9561-6045BD895B5A,09E85092-AF88-EC11-93B0-6045BD91D362,153B9FE0-68BA-E811-80F4-001DD8B72B62,1FE13719-73A4-EC11-983F-00224884C0D3,1FE13719-73A4-EC11-983F-00224884C0D3,1FE13719-73A4-EC11-983F-00224884C0D3,225548F9-8EDA-E711-80EE-001DD8B72B61,225548F9-8EDA-E711-80EE-001DD8B72B61,225548F9-8EDA-E711-80EE-001DD8B72B61,...,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,F39FA0B4-5736-E711-80E6-001DD8B72B61,F39FA0B4-5736-E711-80E6-001DD8B72B61,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62
0,0542DA63-2C64-ED11-9561-6045BD895B5A,1.000000,0.756787,0.770934,0.894875,0.707992,0.719409,0.770934,0.709208,0.785714,...,0.183406,0.190901,0.173591,0.187826,0.166957,0.132599,0.118798,0.100223,0.100223,0.115142
1,09E85092-AF88-EC11-93B0-6045BD91D362,0.756787,1.000000,0.777911,0.743625,0.750120,0.708214,0.777911,0.752325,0.792825,...,0.143940,0.134840,0.136237,0.168468,0.147409,0.117074,0.102748,0.084275,0.084275,0.099586
2,153B9FE0-68BA-E811-80F4-001DD8B72B62,0.770934,0.777911,1.000000,0.757526,0.727754,0.721453,0.792453,0.747696,0.807645,...,0.188526,0.156984,0.158610,0.235973,0.171617,0.170375,0.139558,0.120190,0.120190,0.152171
3,1FE13719-73A4-EC11-983F-00224884C0D3,0.894875,0.743625,0.757526,1.000000,0.817422,0.827586,0.757526,0.696873,0.772049,...,0.180216,0.187581,0.170572,0.205066,0.164053,0.130292,0.116732,0.098480,0.098480,0.129302
4,1FE13719-73A4-EC11-983F-00224884C0D3,0.707992,0.750120,0.727754,0.817422,1.000000,0.817422,0.727754,0.702959,0.743392,...,0.161591,0.189219,0.191180,0.206857,0.165486,0.098573,0.117751,0.099340,0.099340,0.130431


In [39]:
input_person_id = '0542DA63-2C64-ED11-9561-6045BD895B5A'
top_recommendations = df_similarities.nlargest(40,input_person_id)
recommendations = pd.DataFrame({
    'contactID': top_recommendations['contactID'],
    'score': top_recommendations[input_person_id]
})
recommendations = recommendations[recommendations['contactID']!=input_person_id]
recommendations = recommendations.drop_duplicates(subset=['contactID'], keep='first')
print(recommendations.head(10))
# print(input_person_scores[recommendations['contactID']].reset_index().head(10).rename(columns={input_person_id:'score'}))

                               contactID     score
9   236D00C5-A765-ED11-9561-6045BD8956C9  0.954490
21  39DF7E64-2C66-ED11-9561-6045BD895BFB  0.918956
72  EBBF1717-7556-ED11-BBA2-6045BD895BFB  0.910714
49  945F58FA-9A6C-ED11-9561-6045BD895B5A  0.902690
3   1FE13719-73A4-EC11-983F-00224884C0D3  0.894875
60  BA8CB7AB-A2C9-E911-8105-001DD8B72B62  0.855482
17  29B0EB66-82DA-E711-80EE-001DD8B72B61  0.800132
42  8F165595-90DA-E711-80EE-001DD8B72B61  0.792825
69  DE9E5F4B-8349-E811-80F0-001DD8B72B62  0.792825
70  DF8AE390-8349-E811-80F0-001DD8B72B62  0.785905


## test

In [20]:
df3[(df3['contactID']=='0542DA63-2C64-ED11-9561-6045BD895B5A')].head()

,contactID,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,campagneID,campagneType,campagneNaam,campagneSoort,keyphrases,functietitel,functieNaam
0,0542DA63-2C64-ED11-9561-6045BD895B5A,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline,"workshop, plantages, student, pakket, onli...","teamleider graduaatsopleidingen enw bmg, artev...",medewerker


In [21]:
df3[(df3['contactID']=='DE9E5F4B-8349-E811-80F0-001DD8B72B62')].head()

,contactID,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,campagneID,campagneType,campagneNaam,campagneSoort,keyphrases,functietitel,functieNaam
110,DE9E5F4B-8349-E811-80F0-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,08C08CAD-E3EB-E811-80F8-001DD8B72B61,Netwerkevenement,OV-NW-Voka Update: The Big Refresh 6 - voorjaa...,Offline,"workshop, plantages, student, pakket, onli...",unknown,medewerker
114,DE9E5F4B-8349-E811-80F0-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,2D2494E9-0822-E811-80F0-001DD8B72B61,Netwerkevenement,OV-NW-Voka Bilan 2018,Offline,"workshop, plantages, student, pakket, onli...",unknown,medewerker
115,DE9E5F4B-8349-E811-80F0-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,74313A3B-E88D-EA11-810F-001DD8B72B61,Infosessie,OV-Webinar: E-commerce op de Chinese markt,Online,"workshop, plantages, student, pakket, onli...",unknown,medewerker
116,DE9E5F4B-8349-E811-80F0-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,7835D8C0-F488-E811-80F3-001DD8B72B61,Netwerkevenement,OV-NW-Verderkijkers 2018-Think Customer,Offline,"workshop, plantages, student, pakket, onli...",unknown,medewerker
117,DE9E5F4B-8349-E811-80F0-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,BE0C7C7A-373F-E911-80FC-001DD8B72B61,Netwerkevenement,OV-NW-Voka Politica XL,Offline,"workshop, plantages, student, pakket, onli...",unknown,medewerker


In [22]:
df.head().columns

Index(['contactID', 'plaats', 'subregio', 'ondernemingsaard',
       'ondernemingstype', 'activiteitNaam', 'campagneID', 'campagneType',
       'campagneNaam', 'campagneSoort', 'keyphrases', 'functietitel',
       'functieNaam', 'data'],
      dtype='object')

In [23]:
df3

,contactID,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,campagneID,campagneType,campagneNaam,campagneSoort,keyphrases,functietitel,functieNaam
0,0542DA63-2C64-ED11-9561-6045BD895B5A,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline,"workshop, plantages, student, pakket, onli...","teamleider graduaatsopleidingen enw bmg, artev...",medewerker
2,09E85092-AF88-EC11-93B0-6045BD91D362,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,C6D72260-E451-EC11-8C62-000D3ABFCF4A,Netwerkevenement,OV-NW-VokaUpdate-Big Refresh-Voorjaar 2022,Online,"workshop, plantages, student, pakket, onli...",student,medewerker
5,153B9FE0-68BA-E811-80F4-001DD8B72B62,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,7835D8C0-F488-E811-80F3-001DD8B72B61,Netwerkevenement,OV-NW-Verderkijkers 2018-Think Customer,Offline,"workshop, plantages, student, pakket, onli...",marketingverantwoordelijke,medewerker marketing
6,1FE13719-73A4-EC11-983F-00224884C0D3,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline,"workshop, plantages, student, pakket, onli...",community manager & coördinator postgraduaat o...,medewerker
7,1FE13719-73A4-EC11-983F-00224884C0D3,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,4C8FF159-145C-EC11-8F8F-000D3A2BCF4B,Infosessie,OV-JO Breakfastclub april 2022,Offline,"workshop, plantages, student, pakket, onli...",community manager & coördinator postgraduaat o...,medewerker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72039,FC89CE9B-17CA-E711-80EC-001DD8B72B62,ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,24D8FBE3-7BB6-E811-80F4-001DD8B72B62,Netwerkevenement,OV-NW-Voka Ambassadeur - Verkiezing 2018,Offline,"workshop, tijdsinvester, creatief, aankled,...",unknown,"contact lidmaatschap, bedrijfsleider"
72040,FC89CE9B-17CA-E711-80EC-001DD8B72B62,ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,2B01F3C0-E1C4-E911-8104-001DD8B72B61,Netwerkevenement,OV-JO-FinFinder3,Offline,"workshop, tijdsinvester, creatief, aankled,...",unknown,"contact lidmaatschap, bedrijfsleider"
72042,FC89CE9B-17CA-E711-80EC-001DD8B72B62,ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,39BBF8F7-6F3A-E911-80FC-001DD8B72B61,Project,OV-P-Groep J2 - StartUp-Bryo2019,Offline,"workshop, tijdsinvester, creatief, aankled,...",unknown,"contact lidmaatschap, bedrijfsleider"
72055,FC89CE9B-17CA-E711-80EC-001DD8B72B62,ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,52A5376C-6851-E911-80FD-001DD8B72B61,Projectgebonden,OV-P-Plato-Overkoepelende sessie: Veerkracht,Offline,"workshop, tijdsinvester, creatief, aankled,...",unknown,"contact lidmaatschap, bedrijfsleider"
